In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import DBLP_ACM_StructuredBenchmark

benchmark = DBLP_ACM_StructuredBenchmark(data_dir_path="../data/")
benchmark

17:19:35 INFO:Extracting DBLP-ACM-Structured...
17:19:35 INFO:Reading DBLP-ACM-Structured row_dict...
17:19:35 INFO:Reading DBLP-ACM-Structured train.csv...
17:19:35 INFO:Reading DBLP-ACM-Structured valid.csv...
17:19:35 INFO:Reading DBLP-ACM-Structured test.csv...


<DBLP_ACM_StructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/DBLP-ACM/dblp_acm_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'authors', 'venue', 'year']

In [6]:
from tqdm.auto import tqdm
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

  0%|          | 0/4910 [00:00<?, ?it/s]

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_config_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'semantic_title': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'authors': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'venue': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'semantic_venue': {
        'source_attr': 'venue',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'year': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [10]:
from entity_embed import AttrConfigDictParser

row_numericalizer = AttrConfigDictParser.from_dict(attr_config_dict, row_list=benchmark.row_dict.values())
row_numericalizer.attr_config_dict

17:19:35 INFO:For attr=title, computing actual max_str_len
17:19:35 INFO:For attr=title, using actual_max_str_len=18
17:19:35 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:19:39 INFO:For attr=authors, computing actual max_str_len
17:19:39 INFO:For attr=authors, using actual_max_str_len=18
17:19:39 INFO:For attr=venue, computing actual max_str_len
17:19:39 INFO:actual_max_str_len=13 must be even to enable NN pooling. Updating to 14
17:19:39 INFO:For attr=venue, using actual_max_str_len=14
17:19:39 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:19:43 INFO:For attr=year, computing actual max_str_len
17:19:43 INFO:For attr=year, using actual_max_str_len=4


{'title': AttrConfig(source_attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=18, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_title': AttrConfig(source_attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"

In [11]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=32,
    eval_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    row_numericalizer,
    ann_k=ann_k,
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_save_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_save_dir, name=tb_name),
    reload_dataloaders_every_epoch=True
)

GPU available: True, used: True
17:19:44 INFO:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
17:19:44 INFO:TPU available: None, using: 0 TPU cores


In [14]:
trainer.fit(model, datamodule)

17:19:44 INFO:Train pair count: 1332
17:19:44 INFO:Valid pair count: 444
17:19:44 INFO:Test pair count: 444
17:19:44 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 7.6 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
6.5 M     Trainable params
1.1 M     Non-trainable params
7.6 M     Total params
30.285    Total estimated model params size (MB)
17:19:59 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 7.6 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
6.5 M     Trainable params
1.1 M     Non-trainable params
7.6 M     Total params
30.285    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [15]:
model.blocker_net.get_signature_weights()

{'title': 0.26298758387565613,
 'semantic_title': 0.26728543639183044,
 'authors': 0.24771326780319214,
 'venue': 0.09143681824207306,
 'semantic_venue': 0.038239531219005585,
 'year': 0.09233737736940384}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

17:23:54 INFO:Validating best model...


{'valid_f1_at_0.3': 0.3621533442088091,
 'valid_f1_at_0.5': 0.9219562955254943,
 'valid_f1_at_0.7': 0.9854748603351955,
 'valid_f1_at_0.9': 0.9735327963176064,
 'valid_pair_entity_ratio_at_0.3': 2.2612612612612613,
 'valid_pair_entity_ratio_at_0.5': 0.5822072072072072,
 'valid_pair_entity_ratio_at_0.7': 0.5078828828828829,
 'valid_pair_entity_ratio_at_0.9': 0.4786036036036036,
 'valid_precision_at_0.3': 0.22111553784860558,
 'valid_precision_at_0.5': 0.8568665377176016,
 'valid_precision_at_0.7': 0.9778270509977827,
 'valid_precision_at_0.9': 0.9952941176470588,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 0.9977477477477478,
 'valid_recall_at_0.7': 0.9932432432432432,
 'valid_recall_at_0.9': 0.9527027027027027}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

17:23:58 INFO:Train pair count: 1332
17:23:58 INFO:Valid pair count: 444
17:23:58 INFO:Test pair count: 444
17:23:58 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{'test_f1_at_0.3': 0.4206537186167693,
  'test_f1_at_0.5': 0.9269311064718163,
  'test_f1_at_0.7': 0.9844097995545658,
  'test_f1_at_0.9': 0.9816513761467891,
  'test_pair_entity_ratio_at_0.3': 1.8772522522522523,
  'test_pair_entity_ratio_at_0.5': 0.5788288288288288,
  'test_pair_entity_ratio_at_0.7': 0.5112612612612613,
  'test_pair_entity_ratio_at_0.9': 0.481981981981982,
  'test_precision_at_0.3': 0.2663467306538692,
  'test_precision_at_0.5': 0.8638132295719845,
  'test_precision_at_0.7': 0.973568281938326,
  'test_precision_at_0.9': 1.0,
  'test_recall_at_0.3': 1.0,
  'test_recall_at_0.5': 1.0,
  'test_recall_at_0.7': 0.9954954954954955,
  'test_recall_at_0.9': 0.963963963963964}]